# DNA Sequence Classification — Bidirectional LSTM + Transformer
This notebook demonstrates a DNA sequence classification model that combines Bidirectional LSTM and Transformer encoder architecture.

**Steps included:**
- Data loading and preprocessing
- Model architecture (BiLSTM + Transformer)
- Training and evaluation
- Example predictions

In [8]:
# =========================================================
# 🧬 DNA Sequence Classification – BiLSTM + Transformer
# =========================================================
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM, Dense,
    Dropout, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# ------------------------------------------------------
# 🧩 1. Data Augmentation Utilities
# ------------------------------------------------------
def augment_sequence(seq, n=3):
    seqs = []
    for _ in range(n):
        shuffled = ''.join(np.random.permutation(list(seq)))
        seqs.append(shuffled)
    return seqs

def expand_small_classes(df, min_size=10):
    dfs = []
    for label, sub in df.groupby("GeneType"):
        if len(sub) < min_size:
            needed = min_size - len(sub)
            aug = pd.DataFrame({
                "Sequence": sum([augment_sequence(s, n=needed//len(sub)+1) for s in sub["Sequence"]], [])[:needed],
                "GeneType": label
            })
            dfs.append(pd.concat([sub, aug]))
        else:
            dfs.append(sub)
    return pd.concat(dfs).reset_index(drop=True)

# ------------------------------------------------------
# 🧠 2. Model Builder – BiLSTM + Transformer Block
# ------------------------------------------------------
def build_model(vocab_size, num_classes, max_len=200):
    inputs = Input(shape=(max_len,))
    x = Embedding(vocab_size, 64)(inputs)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    attn = MultiHeadAttention(num_heads=2, key_dim=32)(x, x)
    x = LayerNormalization(epsilon=1e-6)(x + attn)
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.3)(x)
    
    if num_classes == 2:
        outputs = Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        outputs = Dense(num_classes, activation='softmax')(x)
        loss = 'sparse_categorical_crossentropy'
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(1e-4), loss=loss, metrics=['accuracy'])
    return model

# ------------------------------------------------------
# 🚀 3. Training Function
# ------------------------------------------------------
def train_model(df, task):
    print(f"\n===== Training {task} =====")
    
    # Expand small classes for stable training
    df = expand_small_classes(df, min_size=20).sample(frac=1, random_state=42).reset_index(drop=True)
    df = df.copy()
    
    # Re-map classes based on task
    if task == 'model2':
        df['GeneType'] = df['GeneType'].apply(lambda x: 'PSEUDO' if x == 'PSEUDO' else 'OTHERS')
    elif task == 'model3':
        df['GeneType'] = df['GeneType'].apply(lambda x: x if x in ['PSEUDO', 'BIOLOGICAL_REGION'] else 'OTHERS')
    elif task == 'model4':
        df['GeneType'] = df['GeneType'].apply(lambda x: x if x in ['PSEUDO','BIOLOGICAL_REGION','ncRNA'] else 'OTHERS')

    # Encode labels
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['GeneType'])

    # Split safely
    stratify = df['label'] if df['label'].value_counts().min() >= 2 else None
    X_train, X_temp, y_train, y_temp = train_test_split(df['Sequence'], df['label'], test_size=0.3, random_state=42, stratify=stratify)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Tokenize
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(X_train)
    max_len = 200
    X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len, padding='post')
    X_val_seq = pad_sequences(tokenizer.texts_to_sequences(X_val), maxlen=max_len, padding='post')
    X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len, padding='post')

    vocab_size = len(tokenizer.word_index) + 1
    num_classes = len(le.classes_)

    # Build & Train
    model = build_model(vocab_size, num_classes)
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model.fit(X_train_seq, y_train, validation_data=(X_val_seq, y_val),
              epochs=15, batch_size=16, callbacks=[es], verbose=2)
    
    # Evaluate
    loss, acc = model.evaluate(X_test_seq, y_test, verbose=0)
    print(f"✅ {task} Test Accuracy: {acc:.4f}")

    # Save
    model.save(f"{task}_bilstm_transformer.keras")
    joblib.dump(tokenizer, f"{task}_tokenizer.pkl")
    joblib.dump(le, f"{task}_le.pkl")

    return model, tokenizer, le

# ------------------------------------------------------
# 🔮 4. Prediction Function
# ------------------------------------------------------
def predict_sequence(sequence, model_path, tokenizer_path, le_path, max_len=200):
    model = load_model(model_path)
    tokenizer = joblib.load(tokenizer_path)
    le = joblib.load(le_path)

    seq = pad_sequences(tokenizer.texts_to_sequences([sequence]), maxlen=max_len, padding='post')
    pred = model.predict(seq)
    
    if pred.shape[1] == 1:  # Binary
        predicted = int(pred[0][0] > 0.5)
    else:  # Multiclass
        predicted = np.argmax(pred, axis=1)[0]
    
    gene_type = le.inverse_transform([predicted])[0]
    print(f"🔬 Predicted GeneType: {gene_type}")
    return gene_type

# ------------------------------------------------------
# 🧪 5. Example Run (Sample Data)
# ------------------------------------------------------
df = pd.DataFrame({
    "Sequence": [
        "ATGCGTACGATCG", "TTTTTACGCGCGT", "GCGCGTATATATA", 
        "CGTATGCGTACGT", "AACCGGTT", "GCGCGCGCGC", "ATATATATAT"
    ],
    "GeneType": [
        "PSEUDO", "BIOLOGICAL_REGION", "ncRNA", "OTHER_TYPE", "PSEUDO", "BIOLOGICAL_REGION", "OTHER_TYPE"
    ]
})

# Train models
model1, tokenizer1, le1 = train_model(df, 'model1')
model2, tokenizer2, le2 = train_model(df, 'model2')
model3, tokenizer3, le3 = train_model(df, 'model3')
model4, tokenizer4, le4 = train_model(df, 'model4')



===== Training model1 =====
Epoch 1/15
4/4 - 7s - 2s/step - accuracy: 0.2143 - loss: 1.9154 - val_accuracy: 0.2500 - val_loss: 1.4717
Epoch 2/15
4/4 - 1s - 127ms/step - accuracy: 0.2857 - loss: 1.7133 - val_accuracy: 0.1667 - val_loss: 1.7154
Epoch 3/15
4/4 - 1s - 126ms/step - accuracy: 0.2679 - loss: 1.7554 - val_accuracy: 0.1667 - val_loss: 1.4616
Epoch 4/15
4/4 - 0s - 123ms/step - accuracy: 0.3393 - loss: 1.6014 - val_accuracy: 0.3333 - val_loss: 1.3836
Epoch 5/15
4/4 - 0s - 110ms/step - accuracy: 0.3393 - loss: 1.7185 - val_accuracy: 0.3333 - val_loss: 1.3803
Epoch 6/15
4/4 - 0s - 110ms/step - accuracy: 0.2857 - loss: 1.6633 - val_accuracy: 0.3333 - val_loss: 1.3637
Epoch 7/15
4/4 - 0s - 104ms/step - accuracy: 0.3036 - loss: 1.5942 - val_accuracy: 0.2500 - val_loss: 1.3964
Epoch 8/15
4/4 - 0s - 109ms/step - accuracy: 0.2321 - loss: 1.6520 - val_accuracy: 0.2500 - val_loss: 1.4322
Epoch 9/15
4/4 - 0s - 104ms/step - accuracy: 0.3750 - loss: 1.4974 - val_accuracy: 0.2500 - val_loss: 

In [17]:
# ------------------------------------------------------
# 🧫 6. Example Prediction
# ------------------------------------------------------
example_seq = "GCCTGTGTGATGATGGAGCTGGGAATACTCTGGGGAGAGAGTCCTCTTTTCAGCTGTATTTTGCTTCCTTCCCACACAGAC"
predict_sequence(example_seq, "model4_bilstm_transformer.keras", "model4_tokenizer.pkl", "model4_le.pkl")


'OTHERS'

In [20]:
# =========================================================
# 🔮 Prediction for All 4 Models
# =========================================================
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

# ----------------------------
# Prediction Helper Function
# ----------------------------
def predict_sequence(sequence, model_path, tokenizer_path, le_path, max_len=200):
    """Load model + tokenizer + encoder and predict GeneType for one DNA sequence."""
    model = load_model(model_path)
    tokenizer = joblib.load(tokenizer_path)
    le = joblib.load(le_path)

    seq = pad_sequences(tokenizer.texts_to_sequences([sequence]), maxlen=max_len, padding='post')
    pred = model.predict(seq, verbose=0)

    # Decode prediction
    if pred.shape[1] == 1:  # Binary model
        predicted = int(pred[0][0] > 0.5)
    else:
        predicted = np.argmax(pred, axis=1)[0]

    gene_type = le.inverse_transform([predicted])[0]
    return gene_type


# ----------------------------
# Predict Using All 4 Models
# ----------------------------
def predict_all_models(sequence):
    results = {}

    for i in range(1, 5):
        model_path = f"model{i}_bilstm_transformer.keras"
        tokenizer_path = f"model{i}_tokenizer.pkl"
        le_path = f"model{i}_le.pkl"

        try:
            prediction = predict_sequence(sequence, model_path, tokenizer_path, le_path)
            results[f"Model {i}"] = prediction
        except Exception as e:
            results[f"Model {i}"] = f"⚠️ Error: {e}"

    print("\n🧬 Prediction Summary for Input Sequence:")
    print("=====================================")
    for model, result in results.items():
        print(f"{model}: {result}")

    return results


# ----------------------------
# 🔬 Example Usage
# ----------------------------
example_seq = "GGGACAGGGGGC"
results = predict_all_models(example_seq)




🧬 Prediction Summary for Input Sequence:
Model 1: BIOLOGICAL_REGION
Model 2: OTHERS
Model 3: OTHERS
Model 4: BIOLOGICAL_REGION
